In [1]:
import cv2
import numpy as np
import os
import time
from collections import defaultdict
import matplotlib.pyplot as plt
import open3d as o3d
from glob import glob
from natsort import natsorted as nat

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [29]:
DEPTH_SCALE = 0.001
MIN_Z, MAX_Z = 0.05, 1.3
def depth_to_colored_pointcloud_world(depth_path, rgb_path, K, D, rvec, tvec, stride=4):
    depth_raw = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
    rgb_img   = cv2.cvtColor(cv2.imread(rgb_path), cv2.COLOR_BGR2RGB)
    if depth_raw is None or rgb_img is None:
        raise FileNotFoundError("Depth or RGB image not found.")

    if depth_raw.dtype != np.float32:
        depth = depth_raw.astype(np.float32) * DEPTH_SCALE
    else:
        depth = depth_raw

    fx, fy = K[0,0], K[1,1]
    cx, cy = K[0,2], K[1,2]
    h, w = depth.shape

    points_world, colors = [], []
    for v in range(0, h, stride):
        for u in range(0, w, stride):
            z = depth[v, u]
            if z < MIN_Z or z > MAX_Z or z <= 0 or np.isnan(z):
                continue
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy
            X_cam = np.array([x, y, z], dtype=np.float32)
            points_world.append(X_cam)
            colors.append(rgb_img[v, u] / 255.0)

    if not points_world:
        return np.empty((0,3), np.float32), np.empty((0,3), np.float32)
    return np.asarray(points_world, np.float32), np.asarray(colors, np.float32)

In [30]:
root_folder="E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard"
# root_folder="../multi_camera_calib/test_0814_2/"
pattern_size = (10, 6)
square_size = 0.025

In [31]:
rgb_paths = []
depth_paths = []
for cam_idx in range(2):
    rgb_paths.append(nat(glob(f"{root_folder}/cam{cam_idx}/rgb_000.jpg")))
    depth_paths.append(nat(glob(f"{root_folder}/cam{cam_idx}/depth_000.png")))
rgb_paths, depth_paths

([['E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam0/rgb_000.jpg'],
  ['E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam1/rgb_000.jpg']],
 [['E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam0/depth_000.png'],
  ['E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam1/depth_000.png']])

In [32]:
K1 = np.load("./src/test/K1.npy")
D1 = np.load("./src/test/D1.npy")
rvecs1 = np.load("./src/test/rvecs1.npy")
tvecs1 = np.load("./src/test/tvecs1.npy")
K2 = np.load("./src/test/K2.npy")
D2 = np.load("./src/test/D2.npy")
rvecs2 = np.load("./src/test/rvecs2.npy")
tvecs2 = np.load("./src/test/tvecs2.npy")

In [33]:
def make_colored_pcd(points, colors):
    pcd = o3d.geometry.PointCloud()
    if points.size > 0:
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

In [36]:
pcds = []
for idx, (rgb_path, depth_path) in enumerate(zip(rgb_paths, depth_paths)):
    print(depth_path[0], rgb_path[0])
    if idx==0:
        K, D, rvec, tvec = K1, D1, rvecs1, tvecs1
        points, colors = depth_to_colored_pointcloud_world(depth_path[0], rgb_path[0], K, D, rvec, tvec, stride=4)
        
        ###
        R1, _ = cv2.Rodrigues(rvecs1)
        R2, _ = cv2.Rodrigues(rvecs2)
        t1 = np.asarray(tvecs1).reshape(3,1)
        t2 = np.asarray(tvecs2).reshape(3,1)
        R21 = R2 @ R1.T
        t21 = t2 - R21 @ t1
        ###
        points_new = points @ R21.T + t21.reshape(3)
        pcd = make_colored_pcd(points_new, colors)
        
    else:
        K, D, rvec, tvec = K2, D2, rvecs2, tvecs2
        points, colors = depth_to_colored_pointcloud_world(depth_path[0], rgb_path[0], K, D, rvec, tvec, stride=4)
        pcd = make_colored_pcd(points, colors)
    pcds.append(pcd)
o3d.visualization.draw_geometries(pcds)
    

E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam0/depth_000.png E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam0/rgb_000.jpg
E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam1/depth_000.png E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam1/rgb_000.jpg


In [37]:
T_new = np.array([[ 0.99939086, -0.00381452,  0.03468956, -0.09666087],
       [ 0.00459311,  0.99973871, -0.02239249,  0.001214  ],
       [-0.03459508,  0.02253818,  0.99914724, -0.0083927 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])
print("hello gina~? What are you doing~~")
print("Oh!  I'm just looking for my new keyboard~~")
print("Ah jin jja?? I'll buy new one for you -sungbeen-")
print("oh.. I'll no sayang.. thank you...")


pcds = []
for idx, (rgb_path, depth_path) in enumerate(zip(rgb_paths, depth_paths)):
    print(depth_path[0], rgb_path[0])
    if idx==1:
        K, D, rvec, tvec = K2, D2, rvecs2, tvecs2
        points, colors = depth_to_colored_pointcloud_world(depth_path[0], rgb_path[0], K, D, rvec, tvec, stride=4)
        
        
        ###
        R1, _ = cv2.Rodrigues(rvecs1)
        R2, _ = cv2.Rodrigues(rvecs2)
        t1 = np.asarray(tvecs1).reshape(3,1)
        t2 = np.asarray(tvecs2).reshape(3,1)
        R21 = R1 @ R2.T
        t21 = t1 - R21 @ t2
        ###
        points_new = points @ R21.T + t21.reshape(3)
        # points_new = points@(((T_new)[:3,:3]).T)+(T_new)[:3,3]
        pcd = make_colored_pcd(points_new, colors)
        
    else: # idx==0
        K, D, rvec, tvec = K1, D1, rvecs1, tvecs1
        points, colors = depth_to_colored_pointcloud_world(depth_path[0], rgb_path[0], K, D, rvec, tvec, stride=4)
        R1 = np.eye(4)[:3,:3]
        t1 = np.eye(4)[:3,3]
        points_new = points @ R1.T + t1.reshape(3)
        pcd = make_colored_pcd(points_new, colors)
    pcds.append(pcd)
o3d.visualization.draw_geometries(pcds)
    

hello gina~? What are you doing~~
Oh!  I'm just looking for my new keyboard~~
Ah jin jja?? I'll buy new one for you -sungbeen-
oh.. I'll no sayang.. thank you...
E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam0/depth_000.png E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam0/rgb_000.jpg
E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam1/depth_000.png E:/Dropbox/real2sim/2025_point/frames/0801_checkerboard/cam1/rgb_000.jpg


In [27]:
R21, t21

(array([[ 0.60493422,  0.02211991, -0.79596815],
        [ 0.02529836,  0.99857556,  0.04697706],
        [ 0.79587347, -0.04855472,  0.60351293]]),
 array([[ 0.47199671],
        [-0.01305847],
        [ 0.20570122]]))